In [1]:
import os.path
import random
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

np.random.seed(1000) # Fixed numpy random seed for reproducible shuffling
torch.manual_seed(1) # set the random seed

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
def select_files(directory, fraction=0.75):
  num_files_to_return =  int(len(os.listdir(directory)) * fraction)
  all_files = [os.path.join(directory, f) for f in os.listdir(directory)[:num_files_to_return] if f.endswith('.jpg')]
  #all_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.jpg')]
  random.shuffle(all_files)
  return all_files[:num_files_to_return]

root = '/content/drive/MyDrive/APS360 Project/APS360 Project Dataset/'
classes = ['Plastic', 'Glass', 'Paper', 'Metal']
folders = [(root+c,c) for c in classes]
main_file_lists = [(select_files(folder,fraction=0.4),label) for (folder,label) in folders]
main_data = [[(torchvision.io.read_image(path),label) for path in folder] for (folder,label) in main_file_lists]

In [4]:
main_data_flat = [item for folder in main_data for item in folder]
main_data_x = [item[0] for item in main_data_flat] #tensors
main_data_y = [item[1] for item in main_data_flat] #labels

In [5]:
class TrashDataset(Dataset):
    """Face Landmarks dataset."""

    #def __init__(self, root_dir, transform=None):
    def __init__(self, inputs,labels, transform=None):
        """
        Arguments:
            root_dir (string): Directory with all the class folders.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.data_x = inputs
        self.data_y = labels
        self.transform = transform


    def __len__(self):
      #classes = ['Plastic', 'Glass', 'Paper', 'Metal']
      #folders = [self.root_dir+c for c in classes]
      #return np.sum([os.listdir(folder) for folder in folders])
      return len(self.data_x)

    def __getitem__(self, idx):
      if self.transform:
          sample = (self.transform(self.data_x[idx]),self.transform(self.data_y[idx]))
          return sample
      sample = (self.data_x[idx],self.data_y[idx])
      return sample

In [6]:
###############################################################################
# Data Loading

def get_relevant_indices(dataset, classes, target_classes):
    """ Return the indices for datapoints in the dataset that belongs to the
    desired target classes, a subset of all possible classes.

    Args:
        dataset: Dataset object
        classes: A list of strings denoting the name of each class
        target_classes: A list of strings denoting the name of desired classes
                        Should be a subset of the 'classes'
    Returns:
        indices: list of indices that have labels corresponding to one of the
                 target classes
    """
    indices = []
    for i in range(len(dataset)):
        # Check if the label is in the target classes
        label_index = dataset[i][1] # ex: 3
        label_class = classes[label_index] # ex: 'cat'
        if label_class in target_classes:
            indices.append(i)
    return indices

def get_data_loader(batch_size):
    """ Loads images of cats and dogs, splits the data into training, validation
    and testing datasets. Returns data loaders for the three preprocessed datasets.

    Args:
        target_classes: A list of strings denoting the name of the desired
                        classes. Should be a subset of the argument 'classes'
        batch_size: A int representing the number of samples per batch

    Returns:
        train_loader: iterable training dataset organized according to batch size
        val_loader: iterable validation dataset organized according to batch size
        test_loader: iterable testing dataset organized according to batch size
        classes: A list of strings denoting the name of each class
    """

    classes = ('Plastic','Metal','Paper','Glass')
    ########################################################################
    # The output of torchvision datasets are PILImage images of range [0, 1].
    # We transform them to Tensors of normalized range [-1, 1].
    transform = None #placeholder
    # Load CIFAR10 training data
    trainset = TrashDataset(main_data_x,main_data_y)
    # Get the list of indices to sample from
    relevant_indices = np.arange(len(main_data_x))

    # Split into train and validation
    np.random.seed(1000) # Fixed numpy random seed for reproducible shuffling
    np.random.shuffle(relevant_indices)
    split = int(len(relevant_indices) * 0.8) #split at 80%

    # split into training and validation indices
    relevant_train_indices, relevant_val_indices = relevant_indices[:split], relevant_indices[split:]
    train_sampler = SubsetRandomSampler(relevant_train_indices)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                               num_workers=0, sampler=train_sampler)
    val_sampler = SubsetRandomSampler(relevant_val_indices)
    val_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                              num_workers=0, sampler=val_sampler)
    """
    # Get the list of indices to sample from
    relevant_test_indices = get_relevant_indices(testset, classes, target_classes)
    test_sampler = SubsetRandomSampler(relevant_test_indices)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                             num_workers=1, sampler=test_sampler)
    """
    return train_loader, val_loader, classes

In [7]:
def transformed_tensor(tensor):
  #temporary, transforms:
  data_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomResizedCrop((256,256)),
    #transforms.CenterCrop((256,256)),
    transforms.ToTensor()#,
    #transforms.Normalize(0, 1)
  ])
  if False: #needs fixing #single image passed in
    return data_transform(tensor)
  else: #tensor of tensors passed in
    return torch.stack([data_transform(t) for t in tensor])

class LargeNet(nn.Module):
    def __init__(self):
        super(LargeNet, self).__init__()
        self.name = "large"
        self.conv1 = nn.Conv2d(3, 5, 5, 2, 1) #convolution layer
        self.pool = nn.MaxPool2d(2, 2) #max pooling layer
        self.conv2 = nn.Conv2d(5, 10, 3)  #convolution layer
        #self.fc1 = nn.Linear(10 * 14 * 14, 32) for 128x128 inputs #linear layer
        self.fc1 = nn.Linear(10 * 30 * 30, 32)
        self.fc2 = nn.Linear(32, 4) #linear layer

    def forward(self, x):
        x = transformed_tensor(x)
        x = self.pool(F.relu(self.conv1(x))) #convolution, relu activation, max pooling
        x = self.pool(F.relu(self.conv2(x))) #convolution, relu activation, max pooling
        #x = x.view(-1, 10 * 14 * 14) #for 128x128 inputs
        x = x.view(-1, 10 * 30 * 30)
        x = F.relu(self.fc1(x)) #linear, relu activation
        x = self.fc2(x) #linear
        #x = x.squeeze(1) # Flatten to [batch_size]
        return x

In [8]:
def evaluate(net, loader, criterion):
    """ Evaluate the network on the validation set.

     Args:
         net: PyTorch neural network object
         loader: PyTorch data loader for the validation set
         criterion: The loss function
     Returns:
         err: A scalar for the avg classification error over the validation set
         loss: A scalar for the average loss function over the validation set
     """
    total_loss = 0.0
    total_err = 0.0
    total_epoch = 0
    for i, data in enumerate(loader, 0):
        inputs, labels = data
        labels = np.array([[1.0 if c == label else 0.0 for c in classes] for label in labels])
        outputs = net(inputs)
        loss = criterion(outputs, torch.tensor(labels))

        probabilities = F.softmax(outputs, dim=0)
        corr = (torch.max(probabilities,dim=1)[1]).squeeze().long() != torch.max(torch.tensor(labels),dim=1)[1]
        total_err += int(corr.sum())
        total_loss += loss.item()
        total_epoch += len(labels)
    err = float(total_err) / total_epoch
    loss = float(total_loss) / (i + 1)
    return err, loss

###############################################################################
# Training Curve
def plot_training_curve(path):
    """ Plots the training curve for a model run, given the csv files
    containing the train/validation error/loss.

    Args:
        path: The base path of the csv files produced during training
    """
    import matplotlib.pyplot as plt
    train_err = np.loadtxt("{}_train_err.csv".format(path))
    val_err = np.loadtxt("{}_val_err.csv".format(path))
    train_loss = np.loadtxt("{}_train_loss.csv".format(path))
    val_loss = np.loadtxt("{}_val_loss.csv".format(path))
    plt.title("Train vs Validation Error")
    n = len(train_err) # number of epochs
    plt.plot(range(1,n+1), train_err, label="Train")
    plt.plot(range(1,n+1), val_err, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Error")
    plt.legend(loc='best')
    plt.show()
    plt.title("Train vs Validation Loss")
    plt.plot(range(1,n+1), train_loss, label="Train")
    plt.plot(range(1,n+1), val_loss, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend(loc='best')
    plt.show()

In [9]:
def train_net(net, batch_size=64, learning_rate=0.01, num_epochs=30):
    ########################################################################
    # Fixed PyTorch random seed for reproducible result
    torch.manual_seed(1000)
    ########################################################################
    # Obtain the PyTorch data loader objects to load batches of the datasets
    train_loader, val_loader, classes = get_data_loader(batch_size)
    ########################################################################
    # Define the Loss function and optimizer
    # The loss function will be Binary Cross Entropy (BCE). In this case we
    # will use the BCEWithLogitsLoss which takes unnormalized output from
    # the neural network and scalar label.
    # Optimizer will be SGD with Momentum.
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)
    #optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    ########################################################################
    # Set up some numpy arrays to store the training/test loss/erruracy
    train_err = np.zeros(num_epochs)
    train_loss = np.zeros(num_epochs)
    val_err = np.zeros(num_epochs)
    val_loss = np.zeros(num_epochs)
    ########################################################################
    # Train the network
    # Loop over the data iterator and sample a new batch of training data
    # Get the output from the network, and optimize our loss function.
    start_time = time.time()
    for epoch in range(num_epochs):  # loop over the dataset multiple times
        total_train_loss = 0.0
        total_train_err = 0.0
        total_epoch = 0
        for i, data in enumerate(train_loader, 0):
            # Get the inputs
            inputs, labels = data
            labels = np.array([[1.0 if c == label else 0.0 for c in classes] for label in labels])
            # Zero the parameter gradients
            optimizer.zero_grad()
            # Forward pass, backward pass, and optimize
            outputs = net(inputs)
            loss = criterion(outputs, torch.tensor(labels))
            #loss = criterion(outputs, labels.float())
            loss.backward()
            optimizer.step()
            # Calculate the statistics

            probabilities = F.softmax(outputs, dim=0)
            corr = (torch.max(probabilities,dim=1)[1]).squeeze().long() != torch.max(torch.tensor(labels),dim=1)[1]
            total_train_err += int(corr.sum())

            total_train_loss += loss.item()
            total_epoch += len(labels)
        train_err[epoch] = float(total_train_err) / total_epoch
        train_loss[epoch] = float(total_train_loss) / (i+1)
        val_err[epoch], val_loss[epoch] = evaluate(net, val_loader, criterion)
        print(("Epoch {}: Train err: {}, Train loss: {} |"+
               "Validation err: {}, Validation loss: {}").format(
                   epoch + 1,
                   train_err[epoch],
                   train_loss[epoch],
                   val_err[epoch],
                   val_loss[epoch]))
        # Save the current model (checkpoint) to a file
        model_path = get_model_name(net.name, batch_size, learning_rate, epoch)
        torch.save(net.state_dict(), model_path)
    print('Finished Training')
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Total time elapsed: {:.2f} seconds".format(elapsed_time))
    # Write the train/test loss/err into CSV file for plotting later
    epochs = np.arange(1, num_epochs + 1)
    np.savetxt("{}_train_err.csv".format(model_path), train_err)
    np.savetxt("{}_train_loss.csv".format(model_path), train_loss)
    np.savetxt("{}_val_err.csv".format(model_path), val_err)
    np.savetxt("{}_val_loss.csv".format(model_path), val_loss)

In [10]:
def get_model_name(name, batch_size, learning_rate, epoch):
    """ Generate a name for the model consisting of all the hyperparameter values

    Args:
        config: Configuration object containing the hyperparameters
    Returns:
        path: A string with the hyperparameter name and value concatenated
    """
    path = "model_{0}_bs{1}_lr{2}_epoch{3}".format(name,
                                                   batch_size,
                                                   learning_rate,
                                                   epoch)
    return path

In [11]:
def plot_training_curve(path):
    """ Plots the training curve for a model run, given the csv files
    containing the train/validation error/loss.

    Args:
        path: The base path of the csv files produced during training
    """
    import matplotlib.pyplot as plt
    train_err = np.loadtxt("{}_train_err.csv".format(path))
    val_err = np.loadtxt("{}_val_err.csv".format(path))
    train_loss = np.loadtxt("{}_train_loss.csv".format(path))
    val_loss = np.loadtxt("{}_val_loss.csv".format(path))
    plt.title("Train vs Validation Error")
    n = len(train_err) # number of epochs
    plt.plot(range(1,n+1), train_err, label="Train")
    plt.plot(range(1,n+1), val_err, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Error")
    plt.legend(loc='best')
    plt.show()
    plt.title("Train vs Validation Loss")
    plt.plot(range(1,n+1), train_loss, label="Train")
    plt.plot(range(1,n+1), val_loss, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend(loc='best')
    plt.show()

In [12]:
large_net = LargeNet() #reset weights
train_net(large_net,num_epochs=20,batch_size=32, learning_rate=0.005)
#large_model_path = get_model_name("large",batch_size=64,learning_rate=0.01,epoch=19) #best so far (not anymore)
#large_model_path = get_model_name("large",batch_size=64,learning_rate=0.008,epoch=19) #random crop first go
large_model_path = get_model_name("large",batch_size=32,learning_rate=0.005,epoch=19)
#large_model_path = get_model_name("large",batch_size=64,learning_rate=0.01,epoch=19)
plot_training_curve(large_model_path)

ZeroDivisionError: division by zero

In [ ]:
%debug

In [23]:
class_counts = {'Plastic':0,'Metal':0,'Glass':0,'Paper':0}
for i, label in enumerate(main_data_y):
  #if i == 140:
    #break
  class_counts[label]+=1

print(class_counts)

{'Plastic': 561, 'Metal': 480, 'Glass': 371, 'Paper': 437}


In [21]:
train_loader, val_loader, c = get_data_loader(1)

In [24]:
class_counts = {'Plastic':0,'Metal':0,'Glass':0,'Paper':0}
for i, data in enumerate(val_loader, 0):
  inputs, labels = data
  for label in labels:
    class_counts[label]+=1
print(class_counts)

{'Plastic': 119, 'Metal': 89, 'Glass': 74, 'Paper': 88}


In [25]:
class_counts = {'Plastic':0,'Metal':0,'Glass':0,'Paper':0}
for i, data in enumerate(train_loader, 0):
  inputs, labels = data
  for label in labels:
    class_counts[label]+=1
print(class_counts)

{'Plastic': 442, 'Metal': 391, 'Glass': 297, 'Paper': 349}


In [16]:
class ann(nn.Module):
  def __init__(self):
    super(ann,self).__init__()
    self.layer1 = nn.Linear(3*128*128,500)
    self.layer2 = nn.Linear(500,80)
    self.layer3 = nn.Linear(80,4)
def forward(self,img):
  flattened = img.view(-1, 3*128*128)
  activation1 = F.relu(self.layer1(flattened))
  activation2 = F.relu(self.layer2(activation1))
  output = self.layer3(activation2)
  return output

In [13]:
large_net = LargeNet() #reset weights
for param in large_net.parameters():
    print(param.shape)

torch.Size([5, 3, 5, 5])
torch.Size([5])
torch.Size([10, 5, 3, 3])
torch.Size([10])
torch.Size([32, 9000])
torch.Size([32])
torch.Size([4, 32])
torch.Size([4])


In [17]:
ann_net = ann() #reset weights
for param in large_net.parameters():
    print(param.shape)

torch.Size([5, 3, 5, 5])
torch.Size([5])
torch.Size([10, 5, 3, 3])
torch.Size([10])
torch.Size([32, 9000])
torch.Size([32])
torch.Size([4, 32])
torch.Size([4])


In [18]:
model_parameters = filter(lambda p: p.requires_grad, large_net.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
params

289004

In [19]:
model_parameters = filter(lambda p: p.requires_grad, ann_net.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
params

24616904